In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.svm import SVR

In [2]:
initial_data = pd.read_csv('./OUTPUT_WBI_exposer_cyclones_v5.csv',sep=';')

In [3]:
initial_data.head()

,SID,NAME,ISO,YEAR,BASIN,SUB BASIN,NATURE,ISO_TIME,COORDS,TOTAL_HRS,...,"Life expectancy at birth, total (years)",Mobile cellular subscriptions (per 100 people),Population density (people per sq. km of land area),Adjusted savings: education expenditure (% of GNI),Rural population (% of total population),Income_level_Final,pop_max_34,pop_max_50,pop_max_64,TOTAL_AFFECTED
0,1949163N07145,DELLA,JPN,1949,WP,WP,TS,"['1949-06-21 03:00:00', '1949-06-22 03:00:00',...","[[35.67, 130.99], [38.35, 135.07], [38.82, 136...",0,...,67.666098,NaN,258.911917,2.867878,36.728,Low,16278929.0,4378623.5,7.321548e+05,194046
1,1950241N23140,JANE,JPN,1950,WP,WP,ET,"['1950-09-02 18:00:00', '1950-09-03 12:00:00',...","[[31.71, 134.12], [38.48, 137.98], [46.7, 152.8]]",6,...,67.666098,NaN,258.911917,2.867878,36.728,Low,39024004.0,14551006.0,4.093306e+06,642117
2,1951224N12316,CHARLIE,JAM,1951,NAm,CS,TS,"['1951-08-17 21:00:00', '1951-08-18 12:00:00']","[[17.34, -75.41], [18.3, -79.4]]",15,...,64.770000,NaN,152.453647,2.578304,66.231,Low,2753687.5,2606976.5,1.067878e+06,20200
3,1951337N09150,AMY,PHL,1951,WP,WP,TS,"['1951-12-08 18:00:00', '1951-12-08 21:00:00',...","[[11.52, 127.35], [11.34, 127.09], [11.17, 126...",15,...,61.105000,NaN,91.065000,2.757732,69.703,Low_Middle,14110069.0,8444818.0,1.750716e+06,60000
4,1952180N05144,EMMA,PHL,1952,WP,WP,TS,"['1952-07-01 18:00:00', '1952-07-01 21:00:00',...","[[9.57, 127.79], [9.63, 127.19], [12.38, 119.4...",3,...,61.105000,NaN,91.065000,2.757732,69.703,Low_Middle,15200242.0,7952954.5,4.470182e+06,103


Trying to estimate the Affected population alone, So removing some features

In [4]:
initial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 49 columns):
SID                                                    1044 non-null object
NAME                                                   1044 non-null object
ISO                                                    1044 non-null object
YEAR                                                   1044 non-null int64
BASIN                                                  1044 non-null object
SUB BASIN                                              1044 non-null object
NATURE                                                 1044 non-null object
ISO_TIME                                               1044 non-null object
COORDS                                                 1044 non-null object
TOTAL_HRS                                              1044 non-null int64
DAY_HRS                                                1044 non-null int64
NIGHT_HRS                                              1044 

In [5]:
req_columns = ['YEAR','BASIN','SUB BASIN','NATURE','TOTAL_HRS','USA_SSHS','ISO_TIME',
              'WIND_CAL_MIN','WIND_CAL_MAX','WIND_CALC_MEAN','PRES_CAL_MIN','PRES_CAL_MAX',
              'PRES_CALC_MEAN','STORM_SPD_MIN','STORM_SPD_MAX','STORM_SPD_MEAN','STORM_DR_MIN',
              'STORM_SPD_MAX','STORM_DR_MEAN','V_LAND_KN','CPI','TOTAL_DAMAGE_(000$)',
               'Population density (people per sq. km of land area)',
              'Rural population (% of total population)','TOTAL_AFFECTED']
req_columns = ['YEAR','BASIN','SUB BASIN','NATURE','TOTAL_HRS','USA_SSHS','ISO_TIME',
              'WIND_CAL_MIN','WIND_CAL_MAX','WIND_CALC_MEAN','PRES_CAL_MIN','PRES_CAL_MAX',
              'PRES_CALC_MEAN','STORM_SPD_MIN','STORM_SPD_MAX','STORM_SPD_MEAN','STORM_DR_MIN',
              'STORM_SPD_MAX','STORM_DR_MEAN','V_LAND_KN',
               'Population density (people per sq. km of land area)',
              'Rural population (% of total population)','TOTAL_AFFECTED']

In [6]:
data = initial_data[req_columns]
data.head(3)

,YEAR,BASIN,SUB BASIN,NATURE,TOTAL_HRS,USA_SSHS,ISO_TIME,WIND_CAL_MIN,WIND_CAL_MAX,WIND_CALC_MEAN,...,STORM_SPD_MIN,STORM_SPD_MAX,STORM_SPD_MEAN,STORM_DR_MIN,STORM_SPD_MAX,STORM_DR_MEAN,V_LAND_KN,Population density (people per sq. km of land area),Rural population (% of total population),TOTAL_AFFECTED
0,1949,WP,WP,TS,0,0,"['1949-06-21 03:00:00', '1949-06-22 03:00:00',...",33.0,55.0,39.133333,...,11,27,19.0,41,27,91.444444,99.82,258.911917,36.728,194046
1,1950,WP,WP,ET,6,-1,"['1950-09-02 18:00:00', '1950-09-03 12:00:00',...",55.6,97.2,76.400000,...,17,30,23.0,13,30,37.666667,99.82,258.911917,36.728,642117
2,1951,NAm,CS,TS,15,1,"['1951-08-17 21:00:00', '1951-08-18 12:00:00']",75.0,102.0,88.500000,...,15,16,15.5,279,16,283.500000,132.40,152.453647,66.231,20200


Extract Month as the cyclone can have seasonal preference

In [7]:
data['MONTH'] = data['ISO_TIME'].apply(lambda x: [int(i[5:7]) for i in eval(x)][0])
data.drop('ISO_TIME', axis=1, inplace=True)

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
data.head(3)

,YEAR,BASIN,SUB BASIN,NATURE,TOTAL_HRS,USA_SSHS,WIND_CAL_MIN,WIND_CAL_MAX,WIND_CALC_MEAN,PRES_CAL_MIN,...,STORM_SPD_MAX,STORM_SPD_MEAN,STORM_DR_MIN,STORM_SPD_MAX,STORM_DR_MEAN,V_LAND_KN,Population density (people per sq. km of land area),Rural population (% of total population),TOTAL_AFFECTED,MONTH
0,1949,WP,WP,TS,0,0,33.0,55.0,39.133333,982.0,...,27,19.0,41,27,91.444444,99.82,258.911917,36.728,194046,6
1,1950,WP,WP,ET,6,-1,55.6,97.2,76.400000,947.5,...,30,23.0,13,30,37.666667,99.82,258.911917,36.728,642117,9
2,1951,NAm,CS,TS,15,1,75.0,102.0,88.500000,972.0,...,16,15.5,279,16,283.500000,132.40,152.453647,66.231,20200,8


In [9]:
data['USA_SSHS'] = data['USA_SSHS'].replace(-5, np.nan)

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
data.pivot_table(values='TOTAL_AFFECTED', index='BASIN')

,TOTAL_AFFECTED
BASIN,
EP,6.089674e+04
NAm,3.026537e+05
NI,1.661712e+06
SI,1.738461e+05
SP,4.268587e+04
WP,7.827592e+05


In [11]:
data.pivot_table(values='TOTAL_AFFECTED', index='SUB BASIN')

,TOTAL_AFFECTED
SUB BASIN,
AS,6.016523e+04
BB,2.432826e+06
CP,1.408350e+04
CS,2.444718e+05
EA,1.986558e+04
EP,6.188004e+04
GM,4.705842e+05
NAm,1.667591e+05
SI,1.991496e+05


In [12]:
data.pivot_table(values='TOTAL_AFFECTED', index='NATURE')

,TOTAL_AFFECTED
NATURE,
DS,47498.125000
ET,271279.315789
MX,542738.500000
NR,668304.596154
SS,57508.250000
TS,583492.447120


In [13]:
## NATURE seems to be ordinal so encoding
from sklearn.preprocessing import LabelEncoder
oe = LabelEncoder()
oe.fit(data['NATURE'])
data['NATURE'] = oe.transform(data['NATURE'])

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
# One Hot encoding for the rest of the categorical variables
data = pd.concat([pd.get_dummies(data[['BASIN', 'SUB BASIN']]), data],axis=1)
data.drop(columns=['BASIN', 'SUB BASIN'],axis=1,inplace=True)

In [15]:
data.head(3)

,BASIN_EP,BASIN_NAm,BASIN_NI,BASIN_SI,BASIN_SP,BASIN_WP,SUB BASIN_AS,SUB BASIN_BB,SUB BASIN_CP,SUB BASIN_CS,...,STORM_SPD_MAX,STORM_SPD_MEAN,STORM_DR_MIN,STORM_SPD_MAX,STORM_DR_MEAN,V_LAND_KN,Population density (people per sq. km of land area),Rural population (% of total population),TOTAL_AFFECTED,MONTH
0,0,0,0,0,0,1,0,0,0,0,...,27,19.0,41,27,91.444444,99.82,258.911917,36.728,194046,6
1,0,0,0,0,0,1,0,0,0,0,...,30,23.0,13,30,37.666667,99.82,258.911917,36.728,642117,9
2,0,1,0,0,0,0,0,0,0,1,...,16,15.5,279,16,283.500000,132.40,152.453647,66.231,20200,8


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 39 columns):
BASIN_EP                                               1044 non-null uint8
BASIN_NAm                                              1044 non-null uint8
BASIN_NI                                               1044 non-null uint8
BASIN_SI                                               1044 non-null uint8
BASIN_SP                                               1044 non-null uint8
BASIN_WP                                               1044 non-null uint8
SUB BASIN_AS                                           1044 non-null uint8
SUB BASIN_BB                                           1044 non-null uint8
SUB BASIN_CP                                           1044 non-null uint8
SUB BASIN_CS                                           1044 non-null uint8
SUB BASIN_EA                                           1044 non-null uint8
SUB BASIN_EP                                           1044 non-null

In [17]:
data.describe()

,BASIN_EP,BASIN_NAm,BASIN_NI,BASIN_SI,BASIN_SP,BASIN_WP,SUB BASIN_AS,SUB BASIN_BB,SUB BASIN_CP,SUB BASIN_CS,...,STORM_SPD_MAX,STORM_SPD_MEAN,STORM_DR_MIN,STORM_SPD_MAX,STORM_DR_MEAN,V_LAND_KN,Population density (people per sq. km of land area),Rural population (% of total population),TOTAL_AFFECTED,MONTH
count,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,...,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1.044000e+03,1044.000000
mean,0.054598,0.225096,0.038314,0.083333,0.057471,0.541188,0.012452,0.025862,0.003831,0.121648,...,16.746169,12.049162,156.434866,16.746169,216.493125,90.180173,316.909952,46.860788,5.756780e+05,7.818008
std,0.227302,0.417846,0.192046,0.276518,0.232852,0.498540,0.110945,0.158800,0.061809,0.327035,...,9.161161,6.571571,127.477415,9.161161,95.485322,31.543540,997.722228,21.914264,1.973459e+06,2.731062
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,1.565217,0.000000,3.000000,2.000000,30.000000,1.786314,0.000000,2.000000e+00,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,11.000000,7.635606,10.000000,11.000000,155.468182,62.700000,49.964096,23.954250,3.000000e+03,7.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,14.000000,10.666667,221.500000,14.000000,260.803571,89.550000,144.583453,53.461000,2.809150e+04,8.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,19.000000,14.456294,276.000000,19.000000,288.333333,114.750000,297.183154,64.807000,2.450182e+05,10.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,69.000000,49.800000,357.000000,69.000000,358.000000,171.800000,20479.769990,92.421000,2.962200e+07,12.000000


In [18]:
data['YEAR'] = 2020 - data['YEAR']

In [24]:
data.shape

(1044, 39)

In [25]:
data=data.dropna()

In [26]:
# Perform min_max_scaler to YEAR, TOTAL_HRS,MONTH
from sklearn.preprocessing import MinMaxScaler, StandardScaler
mm_scaler = MinMaxScaler()
ss_scaler = StandardScaler()

In [27]:
y = data['TOTAL_AFFECTED']
X = data.drop('TOTAL_AFFECTED',axis=1)

In [28]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.1)

In [29]:
columns = list(train_x.columns)
train_x_norm = pd.DataFrame(mm_scaler.fit_transform(train_x),columns = columns)
test_x_norm = pd.DataFrame(mm_scaler.transform(test_x),columns=columns)

In [30]:
train_x_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 38 columns):
BASIN_EP                                               929 non-null float64
BASIN_NAm                                              929 non-null float64
BASIN_NI                                               929 non-null float64
BASIN_SI                                               929 non-null float64
BASIN_SP                                               929 non-null float64
BASIN_WP                                               929 non-null float64
SUB BASIN_AS                                           929 non-null float64
SUB BASIN_BB                                           929 non-null float64
SUB BASIN_CP                                           929 non-null float64
SUB BASIN_CS                                           929 non-null float64
SUB BASIN_EA                                           929 non-null float64
SUB BASIN_EP                                           929 

In [31]:
#standardizin few columns, V_LAND_KN, STORM_DR_MEAN, STORM_SPD_MEAN, STORM_SPD_MAX, 
columns_standard = ['WIND_CAL_MIN','WIND_CAL_MAX','WIND_CALC_MEAN','PRES_CAL_MIN','PRES_CAL_MAX',
                   'PRES_CALC_MEAN','STORM_SPD_MIN','STORM_SPD_MAX','STORM_SPD_MEAN','STORM_DR_MIN',
                   'STORM_DR_MEAN','V_LAND_KN','Population density (people per sq. km of land area)',
                   'Rural population (% of total population)']
for cols in columns_standard:
    train_x_norm[cols] = ss_scaler.fit_transform(train_x_norm[[cols]])
    test_x_norm[cols] = ss_scaler.transform(test_x_norm[[cols]])

In [46]:
output_scaler = StandardScaler()
train_y_norm = output_scaler.fit_transform(train_y.values.reshape(-1,1))
test_y_norm = output_scaler.transform(test_y.values.reshape(-1,1))

- Implementation of algorithms

In [69]:
alg = SVR()

In [70]:
alg.fit(train_x_norm, train_y_norm)

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [71]:
y_pred = alg.predict(test_x_norm)

In [72]:

mean_absolute_error(y_pred, test_y_norm)

0.48618100414070725

In [73]:
mean_squared_error(y_pred, test_y_norm)

2.2588745517519033

In [74]:
alg.score(test_x_norm, test_y_norm)

-0.009094272892728572

In [75]:
r2_score(y_pred, test_y_norm)

-139.70099783306523

In [76]:
param = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),
         'C' : [1,5,10],
         'degree' : [3,8],
         'coef0' : [0.01,10,0.5],
         'gamma' : ('auto','scale')},


In [77]:
from sklearn.model_selection import GridSearchCV

In [78]:
grids = GridSearchCV(alg, param)
grids.fit(train_x_norm, train_y_norm)

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

...

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vamsi/Documents/Programming_projects/wfp_challenge/ven

GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=({'C': [1, 5, 10], 'coef0': [0.01, 10, 0.5],
                          'degree': [3, 8], 'gamma': ('auto', 'scale'),
                          'kernel': ('linear', 'poly', 'rbf', 'sigmoid')},),
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [80]:
grids.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__cache_size': 200,
 'estimator__coef0': 0.0,
 'estimator__degree': 3,
 'estimator__epsilon': 0.1,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
     kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
 'iid': 'deprecated',
 'n_jobs': None,
 'param_grid': ({'kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
   'C': [1, 5, 10],
   'degree': [3, 8],
   'coef0': [0.01, 10, 0.5],
   'gamma': ('auto', 'scale')},),
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 0}

In [81]:
grids.best_estimator_

SVR(C=10, cache_size=200, coef0=0.01, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [82]:
grids.best_score_

0.08803549449171041

In [83]:
y_pred_norm = grids.predict(test_x_norm)

In [84]:
mean_absolute_error(y_pred_norm, test_y_norm)

0.5632516204148417

In [85]:
mean_squared_error(y_pred_norm, test_y_norm)

2.2303917080725553

In [87]:
grids.score(test_x_norm, test_y_norm)

0.003629707024720985